In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
print(GOOGLE_API_KEY[:6])

AIzaSy


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector = embeddings.embed_query("hello, world! I am Twishanu yes")
vector[:5]

c:\Users\aichr\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[-0.027409706264734268,
 -0.010154421441257,
 0.03869868442416191,
 -0.07898290455341339,
 -0.01990700140595436]

In [6]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader

In [7]:
file_path = "D:/AI enabled Test case gen/Documents/fictional_rag_story.pdf"  
if file_path.endswith(".pdf"):
    loader = PyPDFLoader(file_path)
elif file_path.endswith(".docx"):
    loader = Docx2txtLoader(file_path)
else:
    raise ValueError("Unsupported file type")

documents = loader.load()

In [8]:
documents

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-08-15T09:12:08+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-08-15T09:12:08+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'D:/AI enabled Test case gen/Documents/fictional_rag_story.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='The Tale of RAG: Retrieval-Augmented Guardian\nOnce upon a time in the digital city of Algoville, there lived an old librarian named Clara. Clara was\nno ordinary librarian — she managed the Great Knowledge Vault, a magical library where every\npiece of human wisdom was stored, but locked in a language no one could understand. One day, a\nyoung coder named Rian discovered an ancient scroll describing the "Retriever\'s Art" — a method\nto find hidden truths within the vault. With Clara\'s help, he built a mystical chatbot that could\nunderstand the scro

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
chunks = splitter.split_documents(documents)


In [11]:
chunks

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-08-15T09:12:08+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-08-15T09:12:08+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'D:/AI enabled Test case gen/Documents/fictional_rag_story.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='The Tale of RAG: Retrieval-Augmented Guardian'),
 Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-08-15T09:12:08+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-08-15T09:12:08+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'D:/AI enabled Test case gen/Documents/fictional_rag_story.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Once upon a time in the digital city of Algoville, there lived an o

In [12]:
from langchain_community.vectorstores import Chroma

In [14]:
# vectorstore = Chroma(
#     persist_directory="./chroma_sample_doc",
#     embedding_function=embeddings
# )

# batch_size = 10  # you can try 5 or 10 if memory is still low

# for i in range(0, len(chunks), batch_size):
#     batch = chunks[i:i+batch_size]
#     print(f"Processing batch {i // batch_size + 1} / {len(chunks) // batch_size + 1}")
#     vectorstore.add_documents(batch)
#     vectorstore.persist()

# print("✅ Embedding complete and saved!")


vectorstore=Chroma.from_documents(chunks, embedding=embeddings, persist_directory="db")
print("✅ Embedding complete and saved!")

: 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash-lite",convert_system_message_to_human=True)

In [ ]:
print(model.invoke("hi").content)

Hi there! How can I help you today?


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
print(retriever)

tags=['Chroma', 'GoogleGenerativeAIEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000026B7F457FE0> search_kwargs={'k': 3}


In [ ]:
#####
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

In [ ]:
## Both are fed....
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answering_chain = create_stuff_documents_chain(model, chat_prompt)

In [ ]:

rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [ ]:
response = rag_chain.invoke({"input":"what do you summarize from the given doc?"})
response["answer"]

"I need the document you're referring to in order to provide a summary. Please provide the text or a link to the document."